# Aplicamos transfer learning con un modelo que tienen todos los datos menos el de la especie a predecir

In [21]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras 
import os
import json

import random as python_random
import time
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import load_model

from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from keras import layers, models
from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPooling1D

np.random.seed(123)
python_random.seed(123)
tf.random.set_seed(1234)

from funcionesComunes import *

In [14]:
for archivo in os.listdir("models/TCNNMerged_Option2"):

    # Leemos solo los archivos json para obtener los nombres
    if os.path.splitext(archivo)[1] == ".json":
        
        # Obtenemos el nombre de archivo
        nombreArchivo = archivo.split("_best_models.json")[0]

        print(f"PROCESANDO ARCHIVO: {nombreArchivo}")

        data_train = np.load(f'RCPMergedTransfer/{nombreArchivo}_train.npz', allow_pickle=True)
        data_val = np.load(f'RCPMergedTransfer/{nombreArchivo}_val.npz', allow_pickle=True)
        data_test = np.load(f'RCPMergedTransfer/{nombreArchivo}_test.npz', allow_pickle=True)

        X_train = data_train['X_train']
        y_train = data_train['y_train']
        X_val = data_val['X_val']
        y_val = data_val['y_val']
        X_test = data_test['X_test']
        y_test = data_test['y_test']

        # Los valores de normalización estan incluidos en todos los archivos (train, val, test) y cada uno de ellos tiene todo el contenido del resto
        valorNormalizacion = data_train["valorNormalizacion"].item()

        # Cargamos el modelo global
        modelLSTM = tf.keras.models.load_model(f'models/TCNNMerged_Option2/{nombreArchivo}_model_1.keras')

        print(modelLSTM.summary())

        # Obtenemos el valor de batch size
        with open(f'models/TCNNMerged_Option2/{archivo}') as f:
            parameters = json.load(f)

        batch_size_LSTM = parameters[0]["batch_size"]

        historyLSTMTransfer = modelLSTM.fit(X_train, y_train, epochs=200, batch_size=batch_size_LSTM,
                            validation_data=(X_val, y_val), callbacks=[EarlyStopping(monitor='val_loss', patience=20)])
        
        modelLSTM.save(f"models/TCNNMerged_Option2_Trained/{nombreArchivo}.keras")

PROCESANDO ARCHIVO: Pinus wallichiana


/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 34 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_5 (Conv1D)               │ (None, 4, 128)         │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 4, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 2, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 2, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 2, 128)         │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 2, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 2, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 2, 128)         │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 2, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 36)             │         4,644 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            37 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 362,717 (1.38 MB)

...

 Total params: 484,357 (1.85 MB)

 Trainable params: 161,089 (629.25 KB)

 Non-trainable params: 1,088 (4.25 KB)

 Optimizer params: 322,180 (1.23 MB)

None
Epoch 1/200
10020/10020 ━━━━━━━━━━━━━━━━━━━━ 36s 3ms/step - loss: 0.0146 - root_mean_squared_error: 0.1208 - val_loss: 0.0167 - val_root_mean_squared_error: 0.1291
Epoch 2/200
10020/10020 ━━━━━━━━━━━━━━━━━━━━ 34s 3ms/step - loss: 0.0145 - root_mean_squared_error: 0.1205 - val_loss: 0.0184 - val_root_mean_squared_error: 0.1357
Epoch 3/200
10020/10020 ━━━━━━━━━━━━━━━━━━━━ 35s 3ms/step - loss: 0.0145 - root_mean_squared_error: 0.1202 - val_loss: 0.0210 - val_root_mean_squared_error: 0.1450
Epoch 4/200
10020/10020 ━━━━━━━━━━━━━━━━━━━━ 34s 3ms/step - loss: 0.0144 - root_mean_squared_error: 0.1202 - val_loss: 0.0151 - val_root_mean_squared_error: 0.1227
Epoch 5/200
10020/10020 ━━━━━━━━━━━━━━━━━━━━ 34s 3ms/step - loss: 0.0143 - root_mean_squared_error: 0.1196 - val_loss: 0.0260 - val_root_mean_squared_error: 0.1612
Epoch 6/200
10020/10020 ━━━━━━━━━━━━━━━━━━━━ 34s 3ms/step - loss: 0.0144 - root_mean_squared_error: 0.1199 - val_loss: 0.0151 - val_root_mean_squared_error: 0.1229
Epoch 7/200

In [88]:
# Creamos un df que contiene los resultados de los modelos freeze de cada especie
df_Freeze = pd.DataFrame(columns = ["File", "Time", "TrainMSE", "TrainRMSE", "TrainR2", "TrainMAPE", 
                                    "ValidationMSE", "ValidationRMSE", "ValidationR2", "ValidationMAPE",
                                    "TestMSE", "TestRMSE", "TestR2", "TestMAPE"])

for archivo in os.listdir("models/TCNNMerged_Option2"):

    # Leemos solo los archivos json para obtener los nombres
    if os.path.splitext(archivo)[1] == ".json":
        
        # Obtenemos el nombre de archivo
        nombreArchivo = archivo.split("_best_models.json")[0]

        print(f"PROCESANDO ARCHIVO: {nombreArchivo}")

        # Cargamos el modelo entrenado
        df = pd.read_csv(f'RCPMerged/{nombreArchivo}_merged.csv')
        df = df[~df["nametag"].str.startswith("INDI005")]

        # Codificamos, normalización y split de datos
        df = codification(df)
        print(df.shape)

        df, valorNormalizacion = individualNormalization(df)
        print(f"SE HA NORMALIZADO EL ARCHIVO: {archivo}")

        temp_df = df
        train_data, val_data, test_data = split_population_individuals(temp_df, train_pct=0.80, val_pct_in_train=0.20, details=False)
        train_data.shape, val_data.shape, test_data.shape

        # Obtenemos X e y para los datasets de train, val y test 
        WINDOWS_SIZE = 3
        X_train, y_train = df_to_X_y_ind_3(train_data, WINDOWS_SIZE)
        X_val, y_val = df_to_X_y_ind_3(val_data, WINDOWS_SIZE)
        X_test, y_test = df_to_X_y_ind_3(test_data, WINDOWS_SIZE)
        print(X_train.shape, X_test.shape, X_val.shape)

        # Cargamos el modelo global
        if nombreArchivo == "Juniperus spp. ":
            nombreArchivo = "Juniperus spp"
        modelLSTM = tf.keras.models.load_model(f'models/TCNNMerged_Option2_Trained/{nombreArchivo}.keras')
        modelLSTMFreeze = modelLSTM

        # Obtener el optimizador del modelo
        optimizer = modelLSTMFreeze.optimizer

        # Obtenemos el valor de batch size
        with open(f'models/TCNNMerged_Option2/{archivo}') as f:
            parameters = json.load(f)

        batch_size_LSTM = parameters[0]["batch_size"]

        # Indicamos el numero de layers a entrenar
        NUM_TRAINABLE = 1

        numTCNN_layers = sum(1 for layer in modelLSTMFreeze.layers if isinstance(layer, (Conv1D)))

        numFreezeLayers = numTCNN_layers - NUM_TRAINABLE # Congelamos todas menos la útlima capa

        numberLayersFreezed = 0

        # Recorre las capas en orden inverso
        for i, layer in enumerate(modelLSTMFreeze.layers):

            if isinstance(layer, (Conv1D)):
                numberLayersFreezed +=1
        
            if isinstance(layer, (Conv1D, BatchNormalization, MaxPooling1D)) and numFreezeLayers>=numberLayersFreezed:
                
                layer.trainable = False
            
            print(layer, layer.trainable)


        # Compilamos el modelo con los nuevos datos
        modelLSTMFreeze.compile(optimizer=optimizer, loss=MeanSquaredError(), metrics=[
                                    MeanSquaredError(name='mse'),
                                    RootMeanSquaredError(name='rmse'),
                                    MeanAbsolutePercentageError(name='mape')
                                ])

        # Comienza a medir el tiempo de entrenamiento
        start_time = time.time()

        historyLSTMTransfer = modelLSTMFreeze.fit(X_train, y_train, epochs=200, batch_size=batch_size_LSTM,
                                validation_data=(X_val, y_val), callbacks=[EarlyStopping(monitor='val_loss', patience=10)])
        
        # Finaliza la medición del tiempo de entrenamiento
        end_time = time.time()
        print(f"[MODELO CONGELADO]: {end_time-start_time}")

        # Realizar predicciones y calcular métricas para el conjunto de entrenamiento
        predictions_train = predictionForIndividuals(X_train, y_train, modelLSTMFreeze, batch_size_LSTM)
        predictions_train["PredictionsDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_train["ActualDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        train_mse = mean_squared_error(predictions_train["ActualDenormalize"],predictions_train["PredictionsDenormalize"])
        train_rmse = np.sqrt(train_mse)
        train_mape = (np.sum(np.abs(predictions_train["PredictionsDenormalize"] - predictions_train["ActualDenormalize"])) / np.sum(np.abs(predictions_train["ActualDenormalize"]))) * 100
        train_r2 = r2_score(predictions_train["ActualDenormalize"], predictions_train["PredictionsDenormalize"])

        # Realizar predicciones y calcular métricas para el conjunto de validación
        predictions_val = predictionForIndividuals(X_val, y_val, modelLSTMFreeze, batch_size_LSTM)
        predictions_val["PredictionsDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_val["ActualDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        val_mse = mean_squared_error(predictions_val["ActualDenormalize"],predictions_val["PredictionsDenormalize"])
        val_rmse = np.sqrt(val_mse)
        val_mape = (np.sum(np.abs(predictions_val["PredictionsDenormalize"] - predictions_val["ActualDenormalize"])) / np.sum(np.abs(predictions_val["ActualDenormalize"]))) * 100
        val_r2 = r2_score(predictions_val["ActualDenormalize"], predictions_val["PredictionsDenormalize"])

        # Realizar predicciones y calcular métricas para el conjunto de prueba
        predictions_test = predictionForIndividuals(X_test, y_test, modelLSTMFreeze, batch_size_LSTM)
        predictions_test["PredictionsDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_test["ActualDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        test_mse = mean_squared_error(predictions_test["ActualDenormalize"],predictions_test["PredictionsDenormalize"])
        test_rmse = np.sqrt(test_mse)
        test_mape = (np.sum(np.abs(predictions_test["PredictionsDenormalize"] - predictions_test["ActualDenormalize"])) / np.sum(np.abs(predictions_test["ActualDenormalize"]))) * 100
        test_r2 = r2_score(predictions_test["ActualDenormalize"], predictions_test["PredictionsDenormalize"])

        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Train): {train_mse}, {train_rmse}, {train_r2}, {train_mape}")
        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Val): {val_mse}, {val_rmse}, {val_r2}, {val_mape}")
        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Test): {test_mse}, {test_rmse}, {test_r2}, {test_mape}")

        # Guardamos los datos calculados
        df_Freeze.loc[len(df_Freeze)] = [nombreArchivo, end_time-start_time,train_mse, train_rmse, train_r2, train_mape, val_mse, val_rmse, val_r2, val_mape, test_mse, test_rmse, test_r2, test_mape]

df_Freeze.to_csv(f'resultsTransferOption2/resultados_Option2_TCNN.csv', index=False)

PROCESANDO ARCHIVO: Pinus wallichiana
(9365, 77)
SE HA NORMALIZADO EL ARCHIVO: Pinus wallichiana_best_models.json
(5508, 4, 43) (1957, 4, 43) (1500, 4, 43)
<Conv1D name=conv1d_5, built=True> False
<BatchNormalization name=batch_normalization_5, built=True> False
<MaxPooling1D name=max_pooling1d_4, built=True> False
<Dropout name=dropout_1, built=True> True
<Conv1D name=conv1d_6, built=True> False
<BatchNormalization name=batch_normalization_6, built=True> False
<MaxPooling1D name=max_pooling1d_5, built=True> False
<Conv1D name=conv1d_7, built=True> True
<BatchNormalization name=batch_normalization_7, built=True> True
<GlobalAveragePooling1D name=global_average_pooling1d_1, built=True> True
<Dense name=dense_2, built=True> True
<Dense name=dense_3, built=True> True
Epoch 1/200
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0174 - mape: 26.3330 - mse: 0.0174 - rmse: 0.1319 - val_loss: 0.0150 - val_mape: 27.2882 - val_mse: 0.0151 - val_rmse: 0.1226
Epoch 2/200
230/230 ━━━━━━━━━━━━━━━━

/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 4.986299989099817, 2.233002460612128, 0.9260322189000415, 13.043883349882435
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 5.246880380590144, 2.290606989553237, 0.9122350835922024, 15.464001528150199
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 7.116136298363847, 2.6676087228759484, 0.831225766523655, 18.089727467716436
PROCESANDO ARCHIVO: Pinus gerardiana
(10101, 77)
SE HA NORMALIZADO EL ARCHIVO: Pinus gerardiana_best_models.json
(5767, 4, 43) (2181, 4, 43) (1745, 4, 43)
<Conv1D name=conv1d, built=True> False
<BatchNormalization name=batch_normalization, built=True> False
<MaxPooling1D name=max_pooling1d, built=True> False
<Dropout name=dropout, built=True> True
<Conv1D name=conv1d_1, built=True> False
<BatchNormalization name=batch_normalization_1, built=True> False
<MaxPooling1D name=max_pooling1d_1, built=True> False
<Conv1D name=conv1d_2, built=True> True
<BatchNormalization name=batch_normalization_2, built=True> True
<GlobalAveragePooling1D nam

/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 6.414387166299935, 2.532664045289058, 0.8347536811105876, 19.202947935521088
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 9.325058635649649, 3.0536958977032485, 0.7643400760641934, 21.589876042375174
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 7.276594445101172, 2.69751634751324, 0.81887956629712, 19.93298364125001
PROCESANDO ARCHIVO: Betula utilis
(3473, 77)
SE HA NORMALIZADO EL ARCHIVO: Betula utilis_best_models.json
(2059, 4, 43) (665, 4, 43) (593, 4, 43)
<Conv1D name=conv1d_3, built=True> False
<BatchNormalization name=batch_normalization_3, built=True> False
<MaxPooling1D name=max_pooling1d_2, built=True> False
<Conv1D name=conv1d_4, built=True> False
<BatchNormalization name=batch_normalization_4, built=True> False
<MaxPooling1D name=max_pooling1d_3, built=True> False
<Conv1D name=conv1d_5, built=True> False
<BatchNormalization name=batch_normalization_5, built=True> False
<MaxPooling1D name=max_pooling1d_4, built=True> False
<Dropout name=dro

...

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 4.550828839659339, 2.1332671749359804, 0.898615379583831, 22.2114461236428
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 3.137267729691672, 1.7712333922133672, 0.8175593808848911, 24.425837937989712
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 1.2584394833373218, 1.1218018913058232, 0.7684822841532443, 22.69464310507794
PROCESANDO ARCHIVO: Populus ciliata
(1182, 77)
SE HA NORMALIZADO EL ARCHIVO: Populus ciliata_best_models.json
(705, 4, 43) (156, 4, 43) (265, 4, 43)
<Conv1D name=conv1d_5, built=True> False
<BatchNormalization name=batch_normalization_5, built=True> False
<MaxPooling1D name=max_pooling1d_4, built=True> False
<Conv1D name=conv1d_6, built=True> False
<BatchNormalization name=batch_normalization_6, built=True> False
<MaxPooling1D name=max_pooling1d_5, built=True> False
<Dropout name=dropout_1, built=True> True
<Conv1D name=conv1d_7, built=True> False
<BatchNormalization name=batch_normalization_7, built=True> False
<MaxPooling1D name=max_

/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 117.85563927062505, 10.856133716504464, 0.6755011994691803, 28.68850322027755
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 129.9810249927029, 11.400922111509354, 0.4461775534180016, 35.161560879602405
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 66.90509856795892, 8.179553690022392, 0.6656915685847457, 29.356277204818642
PROCESANDO ARCHIVO: Abies pindrow
(16018, 77)
SE HA NORMALIZADO EL ARCHIVO: Abies pindrow_best_models.json
(9042, 4, 43) (3441, 4, 43) (2815, 4, 43)
<Conv1D name=conv1d_5, built=True> False
<BatchNormalization name=batch_normalization_5, built=True> False
<MaxPooling1D name=max_pooling1d_4, built=True> False
<Conv1D name=conv1d_6, built=True> False
<BatchNormalization name=batch_normalization_6, built=True> False
<MaxPooling1D name=max_pooling1d_5, built=True> False
<Dropout name=dropout_1, built=True> True
<Conv1D name=conv1d_7, built=True> False
<BatchNormalization name=batch_normalization_7, built=True> False
<MaxPooling1D name=ma

/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 44.291013837591024, 6.655149422634403, 0.809888818533487, 19.36814414249631
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 34.559356891796966, 5.878720684961735, 0.8028565565437676, 18.69285234851038
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 48.39309158638034, 6.956514327332356, 0.6578745170703524, 19.92465677968514
